In [ ]:
import os
from glob import glob
import warnings
from datetime import datetime

import numpy as np
import pyart
from matplotlib import pyplot as plt
import cftime

import radar_codes
import filtering
import phase
import hydrometeors
import attenuation
import rainrate
import file_util

warnings.simplefilter('ignore')


In [ ]:
#config
vol_root = '/scratch/kl02/vhl548/opol/v2020/ppi/2019'
radar_name = 'opol'
date_str = '20191225'
cf_root = '/g/data/kl02/jss548/PST/polarimetric_rain/cfradial'
rf_root = '/g/data/kl02/jss548/PST/polarimetric_rain/rf3_instant_rainrate'
VERBOSE = False
band = 'C'

In [ ]:
#unpack and list daily zip
vol_ffn_list = sorted(glob(f'{vol_root}/{date_str}/*.nc'))

In [ ]:
#init alpha dictcontinous
alpha_dict = {'z_pairs':[], 'zdr_pairs':[], 'alpha_ts':[], 'dt_ts':[]}

for vol_ffn in vol_ffn_list[100:120]:
    print('processing', vol_ffn)
    
    #read radar volume
    radar = pyart.io.read_cfradial(vol_ffn)
    
    #get time
    valid_time = cftime.num2pydate(radar.time['data'][0], radar.time['units'])
    
    ##################################################################################################
    #
    # Preprocessing
    #
    ##################################################################################################
    
    # Temperature    
    height, temperature, isom = radar_codes.temperature_profile_access(radar)
    radar.add_field('temperature', temperature, replace_existing=True)
    radar.add_field('height', height, replace_existing=True)
    radar.add_field('height_over_isom', isom, replace_existing=True)
    
    ##################################################################################################
    #
    # Retrievals
    #
    ##################################################################################################

    #estimate alpha
    alpha_dict = attenuation.estimate_alpha(radar, alpha_dict, band,
                                           refl_field='corrected_reflectivity', zdr_field='corrected_differential_reflectivity', rhohv_field='cross_correlation_ratio',
                                           verbose=True)

    #estimate specific attenuation
    alpha = alpha_dict['alpha_ts'][-1]
    if VERBOSE:
        print('alpha', alpha)
    radar = attenuation.retrieve_zphi(radar, band, alpha=alpha,
                                     refl_field='corrected_reflectivity', phidp_field='corrected_differential_phase', rhohv_field='cross_correlation_ratio')

    #estimate rainfall
    radar = rainrate.conventional(radar, alpha=92, beta=1.7, refl_field='corrected_reflectivity')
    radar = rainrate.polarimetric(radar, band, refl_field='corrected_reflectivity', kdp_field='corrected_specific_differential_phase', rhohv_field='cross_correlation_ratio')
    
    
    ##################################################################################################
    #
    # Write outputs CF Radial
    #
    ##################################################################################################
    
    #write to cf output
    
    #create paths
    cf_path = f'{cf_root}/{radar_name}/{date_str}'
    if not os.path.exists(cf_path):
        os.makedirs(cf_path)
    cf_fn = f'{radar_name}_{valid_time.strftime("%Y%m%d_%H%M%S")}.vol.nc' #this filename should match
    cf_ffn = f'{cf_path}/{cf_fn}'
    #write to cf
    pyart.io.write_cfradial(cf_ffn, radar)
